In [1]:
!pip3 install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=601c81c93c2f54798e0167618ea1cfb792d6b7f0512a7e1d025c750d328a4e06
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown


In [2]:
!gdown 1JyqoRpXRIfb8czLXQEZhKmzgWLoL9GpM

Downloading...
From: https://drive.google.com/uc?id=1JyqoRpXRIfb8czLXQEZhKmzgWLoL9GpM
To: /workspace/DataSetLiver.zip
100%|██████████████████████████████████████| 2.88G/2.88G [00:49<00:00, 58.6MB/s]


In [3]:
!sudo apt-get -y install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 168 kB of archives.
After this operation, 567 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 unzip amd64 6.0-21ubuntu1.1 [168 kB]
Fetched 168 kB in 0s (336 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package unzip.
(Reading database ... 12682 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-21ubuntu1.1_amd64.deb ...
Unpacking unzip (6.0-21ubuntu1.1) ...
Setting up unzip (6.0-21ubuntu1.1) ...
Processing triggers for mime-support (3.60ubuntu1) ...


In [4]:
!rm -rf dataset
!unzip -qq "DataSetLiver.zip" -d "dataset"

In [5]:
!pip3 -qq install opencv-python

In [6]:
!sudo apt-get install ffmpeg libsm6 libxext6 -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig fontconfig-config fonts-dejavu-core i965-va-driver libaacs0
  libapparmor1 libasound2 libasound2-data libass9 libasyncns0 libavc1394-0
  libavcodec57 libavdevice57 libavfilter6 libavformat57 libavresample3
  libavutil55 libbdplus0 libbluray2 libbs2b0 libcaca0 libcairo2 libcdio-cdda2
  libcdio-paranoia2 libcdio17 libchromaprint1 libcroco3 libcrystalhd3
  libdatrie1 libdc1394-22 libdrm-amdgpu1 libdrm-common libdrm-intel1
  libdrm-nouveau2 libdrm-radeon1 libdrm2 libegl-mesa0 libegl1 libelf1
  libfftw3-double3 libflac8 libflite1 libfontconfig1 libfreetype6 libfribidi0
  libgbm1 libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-bin libgdk-pixbuf2.0-common
  libgl1 libgl1-mesa-dri libglapi-mesa libglvnd0 libglx-mesa0 libglx0 libgme0
  libgomp1 libgraphite2-3 libgsm1 libharfbuzz0b libice6 libicu60 libiec61883-0
  libjack-jackd2-0 libjbig0 libllvm1

In [7]:
#convert json to YOLO
import glob, os, pickle, json
from tqdm import tqdm
from os import listdir, getcwd
from os.path import join
import cv2

classes = ["cystic","FFS","solid"]

def getImagesInDir(dir_path):
  image_list = []
  for filename in glob.glob(dir_path + '/*.jpg'):
    image_list.append(filename)
  return image_list

def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h, class_name):
    # x_center, y_center width heigth
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    return [xmin, ymin, xmax, ymax, class_name]

with open('./dataset/TrainSet/label.json','rb') as file:
  anno_data = json.load(file)

output_dir = './dataset/TrainSet/images/'
input_dir = "./dataset/TrainSet/images/"

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

image_paths = getImagesInDir(input_dir)

for image_path in tqdm(image_paths):
  basename = os.path.basename(image_path)
  basename_no_ext = os.path.splitext(basename)[0]
  out_file = open(output_dir + basename_no_ext + '.txt', 'w')
  im = cv2.imread(image_path)
  [h ,w ,ch] = im.shape
  boxes = anno_data[basename]
  if boxes:
    for bbox_name in boxes:
      bb = xml_to_yolo_bbox(bbox_name,w,h)
      class_name = bbox_name[4]
      class_ind = classes.index(class_name)
      out_file.write(str(class_ind) + " " + " ".join([str(a) for a in bb]) + '\n')
  out_file.close()

print("Finished processing : " + input_dir)

100%|██████████| 13363/13363 [01:02<00:00, 212.12it/s]

Finished processing : ./dataset/TrainSet/images/


In [8]:
!git clone https://github.com/comdet/liver

Cloning into 'liver'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 0), reused 22 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [9]:
%cd liver

/workspace/liver


In [10]:
!sudo apt-get install -y gcc build-essential

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  binutils binutils-common binutils-x86-64-linux-gnu cpp cpp-7 dirmngr
  dpkg-dev fakeroot g++ g++-7 gcc-7 gcc-7-base gnupg gnupg-l10n gnupg-utils
  gpg-agent gpg-wks-client gpg-wks-server gpgsm gpgv libalgorithm-diff-perl
  libalgorithm-diff-xs-perl libalgorithm-merge-perl libasan4 libatomic1
  libbinutils libcc1-0 libcilkrts5 libdpkg-perl libfakeroot
  libfile-fcntllock-perl libgcc-7-dev libisl19 libitm1 libksba8
  liblocale-gettext-perl liblsan0 libmpc3 libmpfr6 libmpx2 libnpth0
  libquadmath0 libstdc++-7-dev libtsan0 libubsan0 make patch pinentry-curses
  xz-utils
Suggested packages:
  binutils-doc cpp-doc gcc-7-locales dbus-user-session libpam-systemd
  pinentry-gnome3 tor debian-keyring g++-multilib g++-7-multilib gcc-7-doc
  libstdc++6-7-dbg gcc-multilib manpages-dev autoconf automake libtool flex
  bison gdb gcc-doc gcc-7-multilib 

In [12]:
!pip3 install -r requirements.txt

  Using cached Pillow-9.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Using cached numpy-1.21.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached opencv_python_headless-4.2.0.34-cp37-cp37m-manylinux1_x86_64.whl (21.6 MB)
     |████████████████████████████████| 106 kB 6.8 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 25.9 MB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 748.8 MB 52 kB/s s eta 0:00:012     |███████████████▊                | 368.8 MB 44.2 MB/s eta 0:00:09
     |████████████████████████████████| 5.9 MB 50.2 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 63.6 MB/s eta 0:00:01
     |██████████████████████████

In [14]:
!pip3 install tensorboard

     |████████████████████████████████| 5.9 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 3.7 MB/s  eta 0:00:01
     |████████████████████████████████| 123 kB 65.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.6 MB 49.7 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 42.5 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 70.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 47.1 MB/s eta 0:00:01
     |████████████████████████████████| 232 kB 96.5 MB/s eta 0:00:01
     |████████████████████████████████| 167 kB 62.3 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 50.2 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 64.4 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [15]:
!python3 train.py --dataDir "/workspace/dataset/TrainSet/images" --numClass "3" --batchSize "64"

Namespace(amp=False, annFile='../manmangogo/train.csv', batchSize=64, bboxCost=5.0, classCost=1.0, clipMaxNorm=0.1, dataDir='/workspace/dataset/TrainSet/images', device='cuda', dimFeedForward=2048, dropout=0.1, eosCost=0.1, epochs=1500, giouCost=2.0, growthRate=32, hiddenDims=512, lr=1e-05, lrBackbone=1e-05, lrDrop=1000, multi=False, numBlocks=[6, 6, 6, 6], numClass=3, numDecoderLayer=6, numEncoderLayer=6, numGroups=8, numHead=8, numQuery=30, numWorkers=8, outputDir='./checkpoint', seed=1588390, targetHeight=608, targetWidth=608, taskName='mango', weight='checkpoint/mango.pt', weightDecay=0.0001)
/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:125: UserWarning: 
NVIDIA GeForce RTX 3090 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70 sm_75.
If you want to use the NVIDIA GeForce RTX 3090 GPU with PyTorch, please check the instructions at https://pytorch.org/get